# <h1 style="text-align: center;">Optimal Control of TCLab using a Gaussian process regression embedded in Pyomo</h1>

<p style="text-align: center;">Alex Dowling<sup>1,a</sup>, Jacob P. Krell<sup>2</sup>, David S. Mebane<sup>2,b</sup>, Kyla Jones<sup>1</sup></p>

<p style="text-align: center;"><sup>1</sup>Department of Chemical and Biomolecular Engineering, University of Notre Dame, Notre Dame, IN 46556, USA <br>
<sup>2</sup>Department of Mechanical and Aerospace Engineering, West Virginia University, Morgantown, WV, 26506-6106, USA</p>

<sup>a</sup>adowling@nd.edu <br>
<sup>b</sup>david.mebane@mail.wvu.edu

---

## Abstract

blah

---

## v4

### Nomenclature

- $\mathbf{GP}_{T_i} \equiv$ GP model of $T_i$
- $\mathbf{GP}_{Q_j} \equiv$ GP model of $Q_j$
- $T_i \equiv$ temperature of port $i$
- $Q_i \equiv$ heater power of port $i$
- $t \equiv$ time elapsed since $t_0$, where $T_i(t_0) = T_{i,\text{ref}}$

### Method

$$\tag{1} i, j\ \exist\ [1, 2]$$

$$\tag{1} \mathbf{GP}_{T_i} = \{T_i\ |\ t,\ Q_1,\ Q_2\}$$

$$\tag{1} \mathbf{GP}_{Q_j} = \{Q_j\ |\ t\}$$

$$\tag{1} T_i = f(t, Q_1(t), Q_2(t))$$

$$\tag{1} \frac{dT_i}{dt} = \frac{\partial T_i}{\partial t} + \left(\frac{\partial T_i}{\partial Q_1}\right) \frac{dQ_1}{dt} + \left(\frac{\partial T_i}{\partial Q_2}\right) \frac{dQ_2}{dt}$$


$$\tag{1a} \begin{align}
C^H_p\frac{dT_{H,1}}{dt} & = U_a(T_{amb} - T_{H,1}) + U_b(T_{S,1} - T_{H,1}) + \alpha P_1u_1\\
\tag{1b} C^S_p\frac{dT_{S,1}}{dt} & = U_b(T_{H,1} - T_{S,1}) 
\end{align}$$

#### Goal

The goal then is to model $\frac{dT_{S,i}}{dt}$ directly, i.e., $\frac{dT_i}{dt}$, better than the ODE. The ODE's second equation is then simply replaced by $\frac{dT_i}{dt}$ calculated from the GP.

Or, use only the first ODE equation (since goal is to get $T_H$) where $T_S$, i.e., $T_i$, is calculated directly from the $\mathbf{GP}_{T_i}$.

The question then becomes how to model either $T_i$ or $\frac{dT_i}{dt}$ to best achieve $T_S$, i.e., $T_i$, predictions in the ODE used to obtain $T_H$.

EFFECTIVELY, GP predictions of $T_i$ (no matter how obtained, but constrained to FoKL-GPy methods / chain rule) need to be better than the ODE result which only uses $T_{ref}$. THEN, the GP model may be placed inside the ODE (using only the first equation, or with second too to PROPAGATE $T_S$ if dT is modeled) for the sole sake of obtaining $T_H$.

---

## Plot of ODE results $T_{1,S}$ for reference


---

## v4a - first attempt to model $T_{1,S}$

---

#### What to do with $T_H$ in order to gauge accuracy of model

pyomo stuff / optimal control

## References

### [ode45](https://www.mathworks.com/help/matlab/ref/ode45.html#bu00_4l_sep_shared-odefun):

Inputs: 
- $(y_1, y_2)_{t_0}$
- $y'_1 = y_1 + 2y_2$
- $y'_2 = 3y_1 + 2y_2$

Output:
- $(y_1, y_2)_t$

### TCLab

- [linear ODE model](https://ndcbe.github.io/controls/notebooks/02.05-Second-Order.html)
- [predictive control using ODE (Optimization-Estimation-Control)](https://ndcbe.github.io/controls/notebooks/07-TCLab-Optimization-Estimation-Control.html)
- [predictive control using ODE (Model-Predictive-Control)](https://ndcbe.github.io/controls/notebooks/06-TCLab-Model-Predictive-Control.html)
- [workshop tutorial on model-based design of experiments](https://dowlinglab.github.io/pyomo-doe/Readme.html)